In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from tensorflow.keras.utils import to_categorical


In [2]:
# Define paths
trainpath = '/Users/amanmaurya/Desktop/Rice Leaf Disease Image Classification/dataset/train'
testpath = '/Users/amanmaurya/Desktop/Rice Leaf Disease Image Classification/dataset/test'

# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)


Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Tungro
Number of images in the folder: 1108
Entering to the folder name: BrownSpot
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Tungro
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)


In [7]:
import numpy as np
import cv2
from tqdm import tqdm  # for progress bar if needed
import math

def compute_gabor_features(image, frequencies=[0.4, 1.6], angles=[0, np.pi/2], grid_size=(4, 4)):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    gabor_features = []
    
    for frequency in frequencies:
        for angle in angles:
            g_kernel = cv2.getGaborKernel((grid_size[0], grid_size[1]), frequency, angle, 5.0, 1.0, 0, ktype=cv2.CV_32F)
            filtered_image = cv2.filter2D(gray_image, cv2.CV_8UC3, g_kernel)
            hist, _ = np.histogram(filtered_image.ravel(), bins=np.arange(0, 256), range=(0, 256))
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-6)  # Normalize the histogram
            gabor_features.extend(hist)
    
    return np.array(gabor_features)

def extract_gabor_features(images, frequencies=[0.4, 1.6], angles=[0, np.pi/2], grid_size=(4, 4)):
    gabor_features = []
    # Initialize tqdm for progress bar if needed
    for img in tqdm(images, desc="Extracting Gabor features"):
        gabor_feature = compute_gabor_features(img, frequencies=frequencies, angles=angles, grid_size=grid_size)
        gabor_features.append(gabor_feature)
    
    return np.array(gabor_features)


In [8]:
train_feature_gabor = extract_gabor_features(X_train)
test_feature_gabor = extract_gabor_features(X_test)

Extracting Gabor features: 100%|██████████| 800/800 [00:03<00:00, 257.90it/s]


# gabor + Random Forest Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test):

        rf = RandomForestClassifier()
        rf = rf.fit(train_feature_gabor, y_train)
        test_pred = rf.predict(test_feature_gabor)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'gabor+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99630542, F1 Score: 0.99497440, AUC: 0.99625000
Run 2 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 1.00000000, F1 Score: 0.99683544, AUC: 0.99687500
Run 3 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99875622, F1 Score: 0.99621200, AUC: 0.99666667
Run 4 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 1.00000000, F1 Score: 0.99683544, AUC: 0.99687500
Run 5 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 1.00000000, F1 Score: 0.99683544, AUC: 0.99687500
Run 6 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 1.00000000, F1 Score: 0.99683544, AUC: 0.99687500
Run 7 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99752475, F1 Score: 0.99559166, AUC: 0.99645833
Run 8 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99630542, F1 Score: 0.99497440, AUC: 0.99625000
Run 9 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 1.00000000, F1 Score: 0.99683544, AUC: 0.99687500
R

# gabor + Decision Tree Classifier

In [10]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test):

        dt = DecisionTreeClassifier()
        dt.fit(train_feature_gabor, y_train)
        test_pred = dt.predict(test_feature_gabor)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'gabor+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.95875000, Recall: 0.95875000, Precision: 0.95890626, F1 Score: 0.95875916, AUC: 0.97250000
Run 2 - Accuracy: 0.94750000, Recall: 0.94750000, Precision: 0.94768704, F1 Score: 0.94755443, AUC: 0.96500000
Run 3 - Accuracy: 0.95500000, Recall: 0.95500000, Precision: 0.95498062, F1 Score: 0.95496078, AUC: 0.97000000
Run 4 - Accuracy: 0.94875000, Recall: 0.94875000, Precision: 0.94887482, F1 Score: 0.94873676, AUC: 0.96583333
Run 5 - Accuracy: 0.94875000, Recall: 0.94875000, Precision: 0.94898808, F1 Score: 0.94865704, AUC: 0.96583333
Run 6 - Accuracy: 0.95500000, Recall: 0.95500000, Precision: 0.95518091, F1 Score: 0.95494556, AUC: 0.97000000
Run 7 - Accuracy: 0.96500000, Recall: 0.96500000, Precision: 0.96498587, F1 Score: 0.96498394, AUC: 0.97666667
Run 8 - Accuracy: 0.95000000, Recall: 0.95000000, Precision: 0.95086115, F1 Score: 0.95002023, AUC: 0.96666667
Run 9 - Accuracy: 0.94875000, Recall: 0.94875000, Precision: 0.94867534, F1 Score: 0.94867997, AUC: 0.96583333
R

# gabor + KNN Classifier

In [11]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test):

        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(train_feature_gabor, y_train)
        test_pred = knn.predict(test_feature_gabor)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'gabor+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 2 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 3 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 4 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 5 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 6 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 7 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 8 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
Run 9 - Accuracy: 0.98000000, Recall: 0.98000000, Precision: 0.98148148, F1 Score: 0.97996795, AUC: 0.98666667
R

# gabor + SVM Classifier

In [12]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test):

        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(train_feature_gabor, y_train_cat)
        test_pred = svm.predict(test_feature_gabor)
        test_pred_proba = svm.predict_proba(test_feature_gabor)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(train_feature_gabor, y_train, test_feature_gabor, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'gabor+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Run 1 - Accuracy: 0.58750000, Recall: 0.58750000, Precision: 0.65606673, F1 Score: 0.54055663, AUC: 0.86541458
Run 2 - Accuracy: 0.58750000, Recall: 0.58750000, Precision: 0.65606673, F1 Score: 0.54055663, AUC: 0.86560000
